<a href="https://colab.research.google.com/github/PUBPOL-2130/notebooks/blob/main/Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 4: the Modifiable Areal Unit Problem (MAUP) and change over time

In [ ]:
!pip -q install maup census

In [ ]:
%config InlineBackend.figure_formats = ["retina"]

import matplotlib.pyplot as plt
import maup
import pandas as pd
import geopandas as gpd

from math import isnan
from census import Census
from collections import Counter

## Introduction to Census blocks

In [ ]:
state_fips = "36"    # New York
county_fips = "047"
county_name = "Kings County"  # Also known as Brooklyn

In [ ]:
census = Census("", year=2020)

In [ ]:
# now we load the block shapefile from the Census website
block_gdf = gpd.read_file(f"https://www2.census.gov/geo/tiger/TIGER2024/TABBLOCK20/tl_2024_{state_fips}_tabblock20.zip")
block_gdf = block_gdf.to_crs("EPSG:2263").set_index("GEOID20")

In [ ]:
# and create a counties and blocks geodataframe
county_block_gdf = block_gdf[block_gdf.COUNTYFP20 == county_fips]

In [ ]:
# plot the blocks of our county, for visual inspection
fig, ax = plt.subplots(figsize=(40, 20))
ax.set_title(f"{county_name} (blocks)", fontsize=18)
ax.axis('off')
county_block_gdf.plot(ax=ax, edgecolor="0.1", linewidth=1, color="#e1f1fd")
plt.axis('off')
plt.show()

### Block-level population data

In [ ]:
# P1 is the TOTPOP table from the Decennial Census -- let's pick out race columns
p1_population_columns = {
    "P1_003N": "white",	      # White alone
    "P1_004N": "black",	      # Black or African American alone
    "P1_005N": "amin",        # American Indian and Alaska Native alone
    "P1_006N": "asian",       # Asian alone
    "P1_007N": "nhpi",        # Native Hawaiian and Other Pacific Islander alone
    "P1_008N": "other",       # Some Other Race alone
    "P1_009N": "two_or_more", # Two or more races
}

In [ ]:
# census.pl is a wrapper around the Decennial (PL) API
block_populations = census.pl.get(
    ("NAME", *p1_population_columns),
    geo={
        "for": "block:*",
        "in": f"county:{county_fips} state:{state_fips}",
    }
)

In [ ]:
# now let's make sure we're using human-readable columns, as chosen above
race_df = pd.DataFrame(block_populations).rename(
    columns={"NAME": "name", **p1_population_columns}
)

In [ ]:
# this pulls those columns into a list called categories
categories = list(p1_population_columns.values())

In [ ]:
# this constructs the 15-digit complete GEOID by concatenating the pieces describing different hierarchical levels
race_df["GEOID20"] = (
    race_df["state"]
    + race_df["county"]
    + race_df["tract"]
    + race_df["block"]
)
race_df = race_df.set_index("GEOID20").drop(columns=["name", "state", "county", "tract", "block"])

In [ ]:
# here we create a new column called "total" that sums over the race categories
race_df["total"] = race_df[categories].sum(axis=1)

In [ ]:
# let's see the dataframe
race_df

In [ ]:
# now we'll create a copy of the dataframe that is based on percents rather than counts
race_with_pcts_df = race_df.copy()

for col in categories:
    race_with_pcts_df[f"{col}_pct"] = (100 * race_df[col] / race_df["total"]).fillna(0)

In [ ]:
race_with_pcts_df

That view shows that some blocks have zero people, while others have hundreds.  Let's get a sense of the largest and smallest.

In [ ]:
# sorts by total population, largest to smallest
race_with_pcts_df.sort_values("total",ascending=False)

In [ ]:
# now we'll create a geodataframe with race and filter out the zero-population blocks
county_block_with_race_gdf = county_block_gdf.join(race_with_pcts_df)
county_block_with_race_populated_gdf = county_block_with_race_gdf[county_block_with_race_gdf.total > 0]

In [ ]:
# here are some choropleth style parameters for the plots below
choropleth_style = dict(
    edgecolor="0.1",
    linewidth=0.2,
    cmap="Blues",
    legend=True,
    legend_kwds={'shrink': 0.4},
)

In [ ]:
fig, ax = plt.subplots(figsize=(40, 20))
ax.axis('off')
ax.set_title(f"{county_name} BPOP/TOTPOP (block level)", fontsize=18)
county_block_with_race_populated_gdf.plot(
    ax=ax,
    column="black_pct",
    vmin=0,
    vmax=100,
    **choropleth_style,
)
plt.show()

## Scale effects: blocks vs. tracts

In [ ]:
tract_gdf = gpd.read_file(f"https://www2.census.gov/geo/tiger/TIGER2024/TRACT/tl_2024_{state_fips}_tract.zip")
tract_gdf = tract_gdf.to_crs("EPSG:2263").set_index("GEOID")

In [ ]:
county_tract_gdf = tract_gdf[tract_gdf.COUNTYFP == county_fips]

We know that tracts are larger, and now let's draw the map to see how much.

In [ ]:
fig, ax = plt.subplots(figsize=(40, 20))
ax.axis('off')
ax.set_title(f"{county_name} (tracts)", fontsize=18)
county_tract_gdf.plot(ax=ax, edgecolor="0.1", linewidth=1, color="#e1f1fd")
plt.axis('off')
plt.show()

In [ ]:
# just to see what is in the dataframe, here's the first row.
county_tract_gdf.iloc[0]

In [ ]:
# to get the tract GEOID, we take digits 1-10 of the full 15-digit identifier
county_block_with_race_gdf["tract"] = county_block_with_race_gdf.index.str.slice(0, 11)

In [ ]:
# now we'll group the block data by tract, to produce tract totals
county_tract_race_df = county_block_with_race_gdf[[*categories, "total", "tract"]].groupby("tract").sum()
county_tract_race_df

In [ ]:
# joining the race data, at the tract level
county_tract_with_race_gdf = county_tract_gdf.join(county_tract_race_df)
county_tract_with_race_gdf

In [ ]:
# recomputing percents at the tract level
for col in categories:
    county_tract_with_race_gdf[f"{col}_pct"] = (100 * county_tract_with_race_gdf[col] / county_tract_with_race_gdf["total"]).fillna(0)

Now when we plot the tract-level choropleth, we expect smoother colors because some of the variation gets averaged out.

In [ ]:
fig, ax = plt.subplots(figsize=(40, 20))
ax.axis('off')
ax.set_title(f"{county_name} Black population % (tracts)", fontsize=18)
county_tract_with_race_gdf.plot(
    ax=ax,
    column="asian_pct",
    vmin=0,
    vmax=100,
    **choropleth_style,
)
plt.show()

Now let's look at the race groups one by one.  The color scale is the same for the blocks and tracts, but it's different from one group to another -- the darkest color is the highest level observed.  

In [ ]:
for col in categories:
    fig, axes = plt.subplots(1, 2, figsize=(15, 8))
    axes[0].axis('off')
    axes[0].set_title(f"{col} % (blocks)", fontsize=18)

    vmax = county_block_with_race_populated_gdf[f"{col}_pct"].quantile(.999)

    county_block_with_race_populated_gdf.plot(
        ax=axes[0],
        column=f"{col}_pct",
        vmin=0,
        vmax=vmax,
        **choropleth_style,
    )

    axes[1].axis('off')
    axes[1].set_title(f"{col} % (tracts)", fontsize=18)
    county_tract_with_race_gdf.plot(
        ax=axes[1],
        column=f"{col}_pct",
        vmin=0,
        vmax=vmax,
        **choropleth_style,
    )

    plt.show()

## Exploring the American Community Survey (ACS)

The ACS has much richer socio-economic variables.  We'll start by making a tract-level dataframe that includes the median age and the median income.

Remember you can find detailed information on ACS variables from the [API documentation](https://www.census.gov/programs-surveys/acs/data/data-via-api.html), the [table shells](https://www.census.gov/programs-surveys/acs/technical-documentation/table-shells.html), or our [Google spreadsheet](https://docs.google.com/spreadsheets/u/1/d/1DtGNarbQLaJdtMiINQ7brQ-Y6zawBkkBkp0VGSENsZw/edit?usp=sharing).

In [ ]:
# Estimate!!Median age --!!Total:
median_age_column = "B01002_001E"

# Estimate!!Median income in the past 12 months --!!Total:
median_income_column = "B06011_001E"

In [ ]:
county_tract_acs_df = pd.DataFrame(
    census.acs5.get(
        (median_age_column, median_income_column),
        geo={
            "for": "tract:*",
            "in": f"county:{county_fips} state:{state_fips}",
        },
        year=2023,
    )
)
county_tract_acs_df["GEOID20"] = (
    county_tract_acs_df["state"]
    + county_tract_acs_df["county"]
    + county_tract_acs_df["tract"]
)
county_tract_acs_df = county_tract_acs_df.set_index("GEOID20").drop(
    columns=["state", "county", "tract"]
).rename(
    columns={
        median_age_column: "median_age",
        median_income_column: "median_income",
    }
)
county_tract_acs_df

Something very strange has happened!  there's a tract with negative values in the millions.  This is sometimes called a "sentinel value," which is dummy data used to flag when something doesn't fit.  (It's an alternative to `NaN`, or "not a number.")

In [ ]:
# how many of these are negative?
print('median age negative in',(county_tract_acs_df["median_age"]<0).sum(),'tracts')
print('median income negative in',(county_tract_acs_df["median_income"]<0).sum(),'tracts')

In [ ]:
county_tract_with_acs_gdf = county_tract_with_race_gdf.join(county_tract_acs_df)

In [ ]:
# we'll use the vmin parameter to avoid plotting those negatives

fig, axes = plt.subplots(1, 2, figsize=(15, 8))
axes[0].axis('off')
axes[0].set_title(f"{county_name}: median age", fontsize=18)

county_tract_with_acs_gdf.plot(
    ax=axes[0],
    column="median_age",
    **choropleth_style,
    vmin=0,
)

axes[1].axis('off')
axes[1].set_title(f"{county_name}: median income ($)", fontsize=18)
county_tract_with_acs_gdf.plot(
    ax=axes[1],
    column="median_income",
    **choropleth_style,
    vmin=0,
)

plt.show()